# DEL 3 - Behandle data fra API-er 



Før vi ser videre på api-et fra Yr, skal vi se litt på behandling, strukturering
og validering av data. 

***

# DEL 3.1 - klasser og dataclass

I denne notebooken skal vi se på hvordan data kan
struktureres med tradisjonelle klasser, og med noe som heter `dataclass` i
python.





In [ ]:
# Importer bibliotekene vi trenger
import requests
import json
from dataclasses import dataclass

# For automatiske tester trenger vi pytest og ipytest
import ipytest
import pytest

ipytest.autoconfig()

***

## 3.1.1: Vanlige klasser og @dataclass

Vi har i forrige del sett på hvordan vi kan bruke `requests` til å hente data
fra Locationforecast APIet til Yr. Nå skal vi se litt på hvordan vi kan
implementere klasser for å gjøre det enklere å bruke APIet til å hente ut data
vi ønsker. Videre ser vi litt på en spesiell type klasse, `dataclass`.

I neste notebook skal vi kikke litt på et bibliotek som heter `pydantic`, som
brukes blant annet til å validere json data fra api-er. Dette biblioteket
utvider funksjonalitet fra vanlige klasser / `dataclass`, så det er nyttig at vi
går gjennom disse to først. 

Under ser du en enkel implementasjon av en klasse for APIet vi har sett på. Gå
gjennom koden og kjør cellen.


In [ ]:
class LocationForecastCompactApi:
    """Klasse for å hente værdata fra Yr.no sitt Locationforecast compact API"""

    def __init__(self, user_agent="TestApp/1.0 (it-test@sanpro.no)"):
        """Initialiserer klassen med basis-URL og headers"""
        self.base_url = "https://api.met.no/weatherapi/locationforecast/2.0/"
        self.base_url_compact = self.base_url + "compact/"
        self.headers = {"User-Agent": user_agent}

    def status(self):
        """Henter statusmelding fra API-et"""
        url = self.base_url + "status"
        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Error fetching status: {response.status_code}")

    def get_forecast(self, latitude, longitude):
        """Henter værdata for gitt bredde- og lengdegrad.

        Args:
            latitude (float): Breddegrad
            longitude (float): Lengdegrad

        Returns:
            dict: Værdata for den angitte posisjonen
        """
        url = self.base_url_compact + f"?lat={latitude}&lon={longitude}"

        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Error fetching forecast: {response.status_code}")


api = LocationForecastCompactApi()
print(api.status())
data = api.get_forecast(59.91, 10.75)
print(json.dumps(data, indent=2))


## Oppgave 3.1-1

- Beskriv kort hva klassen gjør. Ser du noen fordeler med å lage en slik klasse når du skal bruke APIet?
- Hvordan lager du en ny instans / objekt av klassen?
- Hvordan bruker du objektet til å sjekke APIets status?
- Hvordan bruker du objektet til å hente værdata for en gitt lokasjon?


*Skriv svaret ditt her*


---


## 3.1.2: Egendefinerte klasser for å strukturere data

Vi kan fint bruke dictionary-objekter for å lagre sammensatte data i variabler. Om en bruker logiske nøkkelnavn, er det ganske enkelt å finne frem til de verdiene man trenger.

### Eksempel - Kontaktinformasjon

La oss ta et eksempel der vi ønsker å lagre kontaktinformasjon. Vi kan gjøre dette med å lagre verdier i en dictionary:

```python
kontakt = {
    'fornavn': 'Bob',
    'etternavn': 'Sandnes',
    'telefon': 51265854,
    'mobil':  40392284,
    'adresse': 'Langgata 1',
    'postnummer': 4306,
    'poststed': 'Sandnes',
    'epost': 'bob.sandnes@domene.no'
}
```

Når vi da skal finne en verdi som telefonnummer til kontakten, kan vi gjøre
dette med å bruke riktig nøkkelverdi: `kontakt['telefon']`. Dette fungerer godt
på enkle data, og i kortere skript der det er lett å holde oversikten. Når koden
blir større kan det være mer oversiktlig å bruke klasser for å organisere data.
Dette gjør også at en enklere kan validere data, og en får mer hjelp av IDE (som
Visual Studio Code) som gir deg hint til hvilke egenskaper som er tilgjengelig.
Du vil også få beskjed med en gang, mens du skriver, om du for eksempel har
stavet en egenskap feil. Hvis jeg prøver å hente ut telefonnummer med å skrive
`kontakt['tellefon']`, vil dette ikke gi noen feilmelding i IDE, men du får feil
når programmet kjører.

Med bruk av klasser sørger vi også enkelt for at bruk av navn på egenskaper
holder seg likt gjennom hele koden. Kanskje du først lagret e-post-addresser med
nøkkelverdien `e-post` i variabelen `kontaktA`, men senere lagret du e-post med
nøkkelverdi `epost` i variabel `kontaktB`:

```python

kontaktA['e-post'] = 'bob.sandnes@domene.no'

# ... annen kode ...

kontaktB['epost'] = 'kaare@domene.no'

# ... annen kode ...

# Du lagrer begge kontaktvariablene i en kontaktliste
kontakter = [kontaktA, kontaktB]

# Så skal du skrive ut epost til alle kontaktene i listen:
for kontakt in kontakter:
    print(kontakt['epost'])

# Her vil du få en feilmelding, fordi kontaktA brukte nøkkel 'e-post',
# mens kontaktB brukte nøkkel 'epost'
```

La oss se på hvordan dette kunne vært gjort med klasser:


In [ ]:
# Her definerer vi en enkel klasse med de egenskapene vi ønsker for en kontakt
# __init__ metoden kjøres for å initialisere nye objekter av klassen,
# og tar inn parametere som lagres som egenskaper til det nye objektet,
# der self refererer til én spesifikk instans av klassen (ett objekt).
class Kontakt:
    def __init__(
        self,
        fornavn,
        etternavn,
        telefon=None,
        mobil=None,
        adresse=None,
        postnummer=None,
        poststed=None,
        epost=None,
    ):
        self.fornavn = fornavn
        self.etternavn = etternavn
        self.telefon = telefon
        self.mobil = mobil
        self.adresse = adresse
        self.postnummer = postnummer
        self.poststed = poststed
        self.epost = epost


# PS: En liten kommentar om formatering. Over har vi skrevet ett parameter
# per linje i __init__ metoden for å gjøre det mer oversiktlig. Hvis man bare
# har 3-4 parametere i parantesen, kan det fint stå på én linje. Her hadde det
# også vært mulig å skrive hele __init__ metoden på én linje, men den ville
# blitt veldig lang og vanskelig å lese.


# For å lagre kontaktinfo i en variabel, kan vi nå opprette et
# nytt objekt av klassen Kontakt. Vi kan bruke parameternavn
# for å gjøre det tydelig hvilke verdier som settes til hvilke egenskaper.
# Dette gjør koden mer lesbar. Når vi bruker parameternavn, trenger vi
# ikke å følge rekkefølgen på parameterne i __init__ metoden.

kontakt = Kontakt(
    fornavn="Bob",
    etternavn="Sandnes",
    epost="bob.sandnes@domene.no",
    telefon=51265854,
    adresse="Langgata 1",
    postnummer=4306,
    poststed="Sandnes",
    mobil=40392284,
)

# Vi kan også bruke posisjonelle argumenter når vi oppretter et nytt objekt.
# Her bruker vi altså ikke parameternavnene, men følger rekkefølgen på parameterne
# i __init__ metoden. Da blir det mer kompakt, men mindre lesbart, og det kan lettere
# snike seg inn en feil der for eksempel telefon og mobil byttes om.
kontakt2 = Kontakt(
    "Carl",
    "Hansen",
    22334455,
    99887766,
    "Kortveien 2",
    5000,
    "Bergen",
    "carl.hansen@domene.no",
)

# Du så kanskje at noen av parameterne i __init__ metoden har
# standardverdien None. Det betyr at vi kan velge å ikke oppgi
# disse parameterne når vi oppretter et nytt objekt. De vil da
# automatisk få verdien None. Her oppretter vi et nytt objekt
# der vi bare oppgir fornavn og etternavn.
kontakt3 = Kontakt("Dina", "Larsen")

# La oss lagre kontaktene i en liste, og skrive dem ut.
# Legg merke til hvordan vi får tilgang til egenskapene
# til hvert objekt ved å bruke punktum-notasjon.
kontakter = [kontakt, kontakt2, kontakt3]
print("Kontakter:")
for k in kontakter:
    print(f"{k.fornavn} {k.etternavn}, epost: {k.epost}")

## 3.1.3 Dataclass

I standardbiblioteket til python finnes en spesiell type klasse som heter
`dataclass`. Denne er nyttig å bruke til enkle dataklasser, der
initialiseringskoden er inkludert i klassen. Da slipper vi å skrive
`__init__`-metoden, slik som vi gjorde tidligere.

For å bruke `dataclass` trenger vi ikke å installere ekstra bibliotek med `pip`,
men vi må importere biblioteket. Dette er allerede gjort om du har kjørt første
kodecelle i denne notebooken, da vi har inkludert følgende linje:

```python
from dataclasses import dataclass
```


***

## 3.1.4: Et lite sidespor - Decorators

For å bruke `dataclass` må vi benytte oss av noe som heter _decorators_ i
python. En _decorator_ er en slags markør som brukes før en klasse- eller
funksjonsdeklarasjon, og er en linje som starter med symbolet `@`.

En _decorator_ er egentlig en spesiell type funksjon som tar inn en funksjon
eller en klasse, og utvider eller endrer oppførselen til funksjonen/klassen.

La oss lage et kjapt eksempel for å vise hvordan dette fungerer:


In [ ]:
def logg_returverdi(func):
    """Decorator som logger returverdien til en funksjon

    PS: Dette er en såkalt docstring. Den brukes til å dokumentere
    hva funksjonen gjør. Du lager docstrings ved å skrive tekst
    mellom triple anførselstegn. Docstrings kan hentes ut av
    dokumentasjonsverktøy og IDE-er for å vise informasjon om
    funksjonen, så når du fører musen over funksjonsnavnet i en IDE,
    vil du kunne se denne teksten (prøv det!).
    """

    def wrapper(*args, **kwargs):
        """Wrapper-funksjon som logger returverdien til den dekorerte funksjonen.

        Kommentar: *args og **kwargs brukes for å ta imot et vilkårlig antall
        posisjonelle og navngitte argumenter, slik at wrapper-funksjonen kan
        kalle den opprinnelige funksjonen med de samme argumentene. *args samler
        alle posisjonelle argumenter i en tuple, mens **kwargs samler alle
        navngitte argumenter i en dictionary. Hvis vi for eksempel har en
        funksjon definert som def min_funksjon(a, b, c=3), og vi kaller den som
        min_funksjon(1, 2, c=4), vil args få verdien (1, 2) og kwargs få verdien
        {'c': 4}.
        """
        # Kall den opprinnelige funksjonen med de opprinnelige argumentene
        result = func(*args, **kwargs)
        # Logg returverdien til konsollen
        print(f"Funksjonen returnerte: {result}")
        # Returner returverdien, slik at oppførselen til den
        # opprinnelige funksjonen bevares
        return result

    return wrapper


# Nå definerer vi en enkel funksjon som legger sammen to tall,
# og bruker dekoratoren vår til å logge returverdien.
@logg_returverdi
def legg_sammen(a, b):
    """En enkel funksjon som legger sammen to tall og returnerer summen."""
    return a + b


# Kall funksjonen for å se hvordan dekoratoren fungerer
sum_resultat = legg_sammen(5, 7)
print("^^Linjen over ble skrevet av dekorator-funksjonen.^^")
print("\nLa oss skrive ut summen igjen fra hovedprogrammet.")
print("Da ser vi funksjonen legg_sammen fungerer som forventet,")
print("selv om vi har utvidet den med en dekorator:\n")
print(f"Summen er: {sum_resultat}")


Hvis du kjørte cellen over, vil du se at det ble skrevet ut en linje til
konsollen fra dekoratorfunksjonen, i det `legg_sammen` ble kallet, selv om det
ikke er noen kall til `print` i definisjonen til funksjonen `legg_sammen`. Vi
har altså utvidet funksjonaliteten i `legg_sammen` med en egendefinert
dekorator, `@logg_returverdi`.

> Vil du vite mer?
>
> Du kan lese mer om stjerne-uttrykk (`*`) som `*args` og `**kwargs`?
> Les mer om _unpacking_ og operatoren `*` [her](https://www.geeksforgeeks.org/python/starred-expression-in-python/)
> Eller på [W3Schools](https://www.w3schools.com/python/python_args_kwargs.asp)
>
> Hvis du vil se mer på _decorators_ kan du lese om dette [her](https://www.geeksforgeeks.org/python/decorators-in-python/)
> eller på [W3Schools](https://www.w3schools.com/python/python_decorators.asp)


***

## 3.1.5: Kontakt-klassen som `dataclass`

Da har vi sporet av nok, og går tilbake til `dataclass`! La oss redefinere
Kontakt-klassen der vi bruker `dataclass`-dekoratøren:


In [ ]:
@dataclass
class Kontakt:
    fornavn: str
    etternavn: str
    telefon: int = None
    mobil: int = None
    adresse: str = None
    postnummer: int = None
    poststed: str = None
    epost: str = None

Dette var mye mer kompakt! Men vent litt - det er noe ekstra greier her med
`: int ` og `: str`... Dette er noe som heter type hints, men før vi ser på
det, tar vi en liten repetisjon på datatyper og typing i python.


***

## 3.1.6: Litt om datatyper og _Type Hints_


### Typing og datatyper i python

Som du har lært tidligere, er python et _dynamisk typet_ språk, som vil si at
python bestemmer en variabels datatype under kjøring, slik at programmereren
ikke kan bestemme på forhånd, i koden, hvilken datatype en variabel skal ha.
Videre gjøres typingen implisitt, som også kan kalles _duck typing_: "If it
looks like a duck and quacks like a duck, it's a duck". Med andre ord, verdien
som lagres i en variabel bestemmer hvilken datatype variabelen får.

Selv om python er veldig fleksibelt når det kommer til variabler, deklarasjoner
og datatyper, er språket samtidig i bunn og grunn _strongly typed_. Det betyr
at du vil få feilmeldinger når du prøver å gjøre operasjoner på blandede datatyper.
For eksempel vil du få feilmelding om du prøver å legge sammen tekst og tall:

```python
# Følgende vil gi TypeError:
to_pluss_to = "2" + 2

# Python har ingen måte å håndtere denne operasjonen på.
# Rent logisk ser vi også at det ikke finnes noen fornuftig standard her,
# da + mellom to streng-verdier er en append-funksjon, mens + mellom to
# tall er en matematisk operasjon.

# Hvis vi tolker begge verdiene som tekst, vil vi få:
to_pluss_to = "2" + str(2)
# som gir resultatet "22"

# Hvis vi tolker begge verdiene som tall, vil vi få:
to_pluss_to = int("2") + 2
# som gir resultatet 4
```

> Kommentar:
>
> Hvordan ser _weak typing_ ut?
>
> For eksempel kan man i JavaScript, som er svakt typet,
> blande tekst og tall. Da blir en av datatypene automatisk konvertert.
> Så der kan vi skrive `to_pluss_to = "2" + 2`
> uten at dette gir feilmelding, og vil få resultatet `"22"`
> (2 ble konvertert til tekst). For `-` vil tekst bli konvertert til tall,
> så `to_minus_to = "2" - 2` får resultat 0.
>
> Dette kan være praktisk i noen sammenhenger, men kan også lede til
> uventede resultater og feil som blir vanskelige å spore.

Selv om python er _strongly typed_ vil du kunne blande noen typer uten å
konvertere dem først. Dette gjelder spesielt i sammenheng med tallrelaterte
datatyper, som `int` og `float`, der også `bool` kan blandes med tall-typer da
`bool` faktisk er en underklasse av `int`. Så når du jobber med tall i python
vil datatyper endres dynamisk som vist under:

```python
# int + float gir float
resultat = 5 + 2.5      # 7.5 (float)

# int / int, der resultatet ikke er et heltall, gir float
resultat = 10 / 3       # 3.333... (float)

# operasjoner med float, som resulterer i et heltall,
# vil fortsatt resultere i datatypen float
resultat = 7.5 / 2.5      # 3.0 (float)
resultat = 2.0 / 2        # 1.0 (float)
resultat = 2.0 + 2        # 4.0 (float)

# bool og int
resultat = True + 5     # 6
resultat = False * 10   # 0


# Blanding i logiske operasjoner:

# Like tall med ulike datatyper aksepteres
5 == 5.0               # True
True == 1              # True

# Fortsatt ikke mulig å sammenligne tall som tekst med tall
"5" == 5               # False
```

Konverteringen følger hierarkiet `bool → int → float (→ complex)`. Det vil si at
`bool` kan automatisk konverteres til `int` og `int` til `float`, men `float`
vil aldri konverteres automatisk til `int` osv. (`complex` er datatype for
komplekse tall, som er en spesiell type tall i matematikken).

Python følger altså en ganske streng typing, men noen veldefinerte og forutsigbare
automatiske typekonverteringer gjøres som vist over. Andre språk kan være strengere
når det kommer til datatyper, der for eksempel `5 / 2` vil gi `2` (heltallsdivisjon).
Noen språk vil da kunne tillate `5.0 / 2` som gir flyttallet `2.5`, mens veldig strengt
typede språk vil gi feilmelding ved `5.0 / 2` og bare tillate uttrykk som ikke blander
flyttall med heltall, som `5.0 / 2.0`.

Så hva gjør du når du vil blande datatyper, eller ha mer kontroll over resultatet fra
operasjoner med ulike datatyper? Kanskje du allerede vet svaret? Du bruker spesielle
operatorer eller eksplisitt konvertering:

```python
# Konverter datatyper før operasjonen gjennomføres:
resultat = int("5") + 3      # 8 (int)
resultat = float("5") + 3    # 8.0 (float)
resultat = int("5") + 3.0    # 8.0 (float)

# Obs - du vil få feilmelding dersom du prøver å gjøre en
# ugylding konvertering
resultat = int("5.0") + 3    # ValueError ("5.0" kan bare konverteres til float)
resultat = float("5,0") + 3  # ValueError (python gjenkjenner bare '.' som desimaltegn)

# Heltallsdivisjon - noen ganger ønsker vi ikke at resultatet
# av en divisjon skal resultere i et flyttall. Da kan vi bruke
# operatoren // som gjennomfører en heltallsdivisjon:
resultat = 5 // 2            # 2 (int)
# Eller vi kan konvertere resultatet til int:
resultat = int(5 / 2)        # 2 (int)
# Obs - siden ene tallet er flyttall, vil resultatet få datatypen float,
# selv om heltallsdivisjon brukes:
resultat = 5.0 // 2          # 2.0 (float)
```

> Vil du vite mer?
>
> Les mer om:
>
> - [Duck typing](https://www.geeksforgeeks.org/python/duck-typing-in-python/)
> - [Typing in python - strong, dynamic, implicit](https://medium.com/@pavel.loginov.dev/typing-in-python-strong-dynamic-implicit-c3512785b863)
> - [w3 - Data Types](https://www.w3schools.com/python/python_datatypes.asp)
> - [w3 - Python Casting](https://www.w3schools.com/python/python_casting.asp)


### _Type Hints_

Dynamisk typing gir stor fleksibilitet og mange fordeler. Samtidig blir det
mindre synlig i koden hvilke datatyper det forventes at ulike variabler skal ha.
Selv om typing i python skjer dynamisk, betyr ikke dette at det er uviktig hvilke
datatyper som brukes! Og om variabler og parametre mottar data av feil type, vil
dette raskt lede til feil og krasj av programmet!

For å gjøre koden mer lesbar, der det er enkelt å se hvilke datatyper som
forventes, samt gjøre feilsøking enklere, kan man i python bruke noe som heter
_Type Hints_. Det er en god grunn til at ordet _hint_ er i navnet -
python-interpreteren (programmet som tolker og kjører koden din) ignorerer type
hints fullstendig. Type hints blir altså aldri lest av datamaskinen, og er kun
til støtte for utviklere.

La oss ta et eksempel på en funksjon som beregner arealet av en firkant. Først
har vi funksjonen her, uten type hints:

```python
def beregn_areal(lengde, bredde):
    return lengde * bredde

resultat = beregn_areal(5, 10)
```

Under defineres samme funksjon i en kodecelle, med type hints:


In [ ]:
def beregn_areal(lengde: float, bredde: float) -> float:
    """Beregner arealet av et rektangel gitt lengde og bredde.

    Kommentar
    ---
    Her bruker vi type hints for å angi at begge parametrene,
    lengde og bredde, skal være av typen float, og at funksjonen
    returnerer en float-verdi.
    """
    return lengde * bredde


resultat: float = beregn_areal(5.0, 10.0)  # Type hints kan også brukes for variabler

print(f"Arealet er: {resultat}")

Nå gir vi mer informasjon til utviklere, og til de som skal bruke funksjonen
(som kan være deg selv eller andre). Koden blir mer lesbar, og det er raskere
å vite nøyaktig hvilke datatyper det forventes at funksjonen kan håndtere,
og hvilken datatype som vil returneres av funksjonen.

Når du fører musen over funksjonsnavnet, eller etter du skriver funksjonsnavnet
og `(` i IDE, vil du få opp en boks med informasjon om funksjonen. Denne boksen
inkluderer funksjonens header (f.o.m. `def` t.o.m. `:`). For funksjonen vår vil
vi se:

```python
(function) def beregn_areal(
    lengde: float,
    bredde: float
) -> float
```

Merk notasjonene:

- `:` etter parameter- eller variabelnavn, etterfulgt av navnet på typen.
- `->` mellom funksjonens sluttparantes `)` og `:`, etterfulgt av navnet på
  typen til returverdien.

### Hva skjer om vi ikke følger type hints?

La oss prøve!


In [ ]:
# Her sender vi int i stedet for float
resultat = beregn_areal(5, 10)
print(f"Arealet er (med int): {resultat}")

# Her sender vi en streng i stedet for float
try:
    resultat = beregn_areal("5", "10")
    print(f"Arealet er (med str): {resultat}")
except TypeError as e:
    print(f"Feil: {e}")

# Og et par pussige eksempler
resultat = beregn_areal("5", 10)
print(f"Arealet er (med str og int): {resultat}")

resultat = beregn_areal([5], 10)
print(f"Arealet er (med liste og int): {resultat}")

## Oppgaver (3.1-2 -> 3.1-3) - datatyper og Type Hints

### Oppgave 3.1-2

Forklar resultatene etter du kjørte cellen over:

- Hva skjedde med returtypen når begge parametrene var `int`?
- Hva skjenne når begge parametrene var av typen `str`?
  - Hvilken eksakt linje i koden utløste feilen (bruk debug om du er usikker)?
- Hva skjedde når første var `str` og andre var `int`
  - Hvorfor?
- Hva skjedde når første var `list` og andre var `int`
  - Hvorfor?


_Dobbeltklikk her og skriv inn ditt svar_


Som nevnt bryr ikke datamaskinen seg om type hints! Derfor kjøres koden som om
det ikke var type hints i koden..

Som demonstrert tidligere gav ikke type hint noe validering av de faktiske
verdiene funksjonen mottok. I tillegg ble kun `float` angitt, selv om funksjonen
også håndterer `int`. La oss se på et litt mer robust eksempel:


In [ ]:
def beregn_areal(lengde: float | int, bredde: float | int) -> float | int:
    """Beregner arealet av et rektangel gitt lengde og bredde.

    Kommentar
    ---
    Vi kan bruke union types (float | int) for å angi at typen kan være
    en av de angitte typene, adskilt med en vertikal strek (|).
    """
    # For å gjøre funksjonen mer robust, kan vi sjekke
    # at parametrene har gyldige typer før vi utfører beregningen.
    if not isinstance(lengde, (float, int)):
        raise TypeError(f"Lengde må være av typen float eller int, ikke {type(lengde)}")
    if not isinstance(bredde, (float, int)):
        raise TypeError(f"Bredde må være av typen float eller int, ikke {type(bredde)}")

    return lengde * bredde


# Her er det egentlig unødvendig å bruke type hints
# for resultat-variabelen, fordi IDE automatisk skjønner
# at typen skal settes til float | int basert på returtypen
# til funksjonen. Før musen over resultat-variabelen for å se
# at VSCode skjønner hvilken type variabelen forventes å ha.
resultat = beregn_areal(5, 10)

print(f"Arealet er: {resultat}")

# La oss teste igjen med ugyldige typer
try:
    resultat = beregn_areal("5", "10")
except TypeError as e:
    print(f"Feil: {e}")

try:
    resultat = beregn_areal(5, "10")
except TypeError as e:
    print(f"Feil: {e}")

try:
    resultat = beregn_areal([5], 10)
except TypeError as e:
    print(f"Feil: {e}")

Siden ikke python-interpreter leser type hints, kan det være aktuelt at du
skriver kode som sjekker at parametrene som sendes inn i funksjonen er gyldige.
Dette har vi gjort her ved å sjekke datatypen til parametrene, og reise en
egendefinert feilmedling dersom funksjonen mottar en uventet datatype. Det er
ikke helt nødvendig å gjøre dette, og det vil være en vurdering i det konkrete
tilfellet om det er lurt å implementere slik parameterkontroll.

Robust kode med presis feilhåndtering er imidlertid god kodepraksis, og med
større og mer komplekse programmer kan det fort bli utfordrende å spore feil
dersom man ikke implementerer sjekker og reiser beskrivende feil med forklarende
feilmeldinger. Se eksempel på et enkelt program som bruker `beregn_areal`:


In [ ]:
def beregn_areal(lengde: float | int, bredde: float | int) -> float | int:
    """Beregner arealet av et rektangel gitt lengde og bredde.

    Kommentar
    ---
    Her har vi type-hints, men har droppet sjekkene for
    parameter-typene.
    """
    return lengde * bredde


def beregn_areal_robust(lengde: float | int, bredde: float | int) -> float | int:
    """Beregner arealet av et rektangel gitt lengde og bredde.

    Kommentar
    ---
    Her er den mer robuste versjonen vist tidligere,
    med sjekk av parameter-typene.
    """
    if not isinstance(lengde, (float, int)):
        raise TypeError(f"Lengde må være av typen float eller int, ikke {type(lengde)}")
    if not isinstance(bredde, (float, int)):
        raise TypeError(f"Bredde må være av typen float eller int, ikke {type(bredde)}")

    return lengde * bredde


def beregn_areal_fra_input(beregn_areal=beregn_areal):
    """Beregner arealet av et rektangel basert på brukerinput."""
    print("--- Beregn areal for rektangel ---\n")
    bredde = int(input("Skriv inn bredde:"))
    lengde = input("Skriv inn lengde:")

    areal = beregn_areal(lengde, bredde)
    print(f"\nArealet av rektangel med lengde {lengde} og bredde {bredde} er:\n{areal}")

Over har vi definert `beregn_areal`, der vi ikke har implementert sjekk av
parameter-typene. `beregn_areal_robust` har de samme typesjekkene som over. Funksjonen `beregn_areal_fra_input` kjører et kort skript som
skal ta inn brukerdata (lengde og bredde) med `input()`, og skrive ut arealet
av et rektangel med slik lengde og bredde. I funksjonen er det sneket seg inn en liten
feil, som gjør at vi får et litt interessant resultat...

Kjør cellen over, og så den under som simulerer brukerinput. Det er ikke så viktig å se på eller forstå koden i cellen under, fokuser på output fra cellen etter du kjører den.


In [ ]:
%%ipytest -s


def mocked_input(input_iter, prompt):
    svar = next(input_iter)
    print(f"{prompt.strip()} {svar}")
    return svar


def test_beregn_areal_fra_input_ok(monkeypatch):
    print("\n#### Kjører test med beregn_areal, inndata '1' og '5' ####\n")
    input_iter = iter(["1", "5"])
    monkeypatch.setattr(
        "builtins.input", lambda prompt: mocked_input(input_iter, prompt)
    )
    beregn_areal_fra_input()
    print("\n-----\nKommentar: Dette ser jo riktig ut?")


def test_beregn_areal_fra_input(monkeypatch):
    print("\n#### Kjører test med beregn_areal, inndata '10' og '5' ####\n")
    input_iter = iter(["10", "5"])
    monkeypatch.setattr(
        "builtins.input", lambda prompt: mocked_input(input_iter, prompt)
    )
    beregn_areal_fra_input()
    print(
        "\n-----\nKommentar: Her er det noe som ikke stemmer - men vi får ingen feilmelding!"
    )


def test_beregn_areal_fra_input_robust(monkeypatch):
    print("\n#### Kjører test med beregn_areal_robust, inndata '10' og '5' ####\n")
    input_iter = iter(["10", "5"])
    monkeypatch.setattr(
        "builtins.input", lambda prompt: mocked_input(input_iter, prompt)
    )
    with pytest.raises(TypeError) as exc_info:
        beregn_areal_fra_input(beregn_areal=beregn_areal_robust)
    print("\n---------------\nProgrammet ble avsluttet.\nFølgende TypeError ble reist:")
    print(exc_info.value)
    print("Stack trace:")
    for trace in exc_info.traceback:
        print(trace)
    print(
        "\n-----\nKommentar: Her får vi en feilmelding som forteller oss hva som gikk galt!"
    )


Over skal du se output fra tre tester. Først med bruker-input 1 og 5, så med brukerinput 10 og 5, og til slutt brukerinput 10 og 5 med den robuste implementeringen.

> Merk:
>
> Det kan være du må trykke på lenken _scrollable element_ for å se hele utskriften over

### Oppgave 3.1-3

Tenk gjennom følgende og svar i tekstboksen under:

- Hvorfor ser svaret riktig ut i den første testen?
- Hvorfor blir svaret feil i den andre testen
  - Hva kan være utfordringene med å spore denne feilen?
- Hvorfor får vi feilmelding på den siste testen?
  - Hvordan gjør feilmeldingen det enklere å oppdage og spore feilen?


_Dobbeltklikk her og skriv inn ditt svar_


In [ ]:
# Kjør denne cellen om du vil teste hvordan beregn_areal_fra_input UTEN robust
# typesjekking fungerer, der du kan skrive inn verdier for lengde og bredde
# selv.
beregn_areal_fra_input()

In [ ]:
# Kjør denne cellen om du vil teste hvordan beregn_areal_fra_input MED robust
# typesjekking fungerer, der du kan skrive inn verdier for lengde og bredde
# selv.
beregn_areal_fra_input(beregn_areal=beregn_areal_robust)

***

## 3.1.7: Tilbake til `dataclass`

Da har vi tatt en litt lengre omvei innom datatyper og type hints, og det er på tide at vi kikker videre på `dataclass`. Under er klassen som vi så vidt så på tidligere:


In [ ]:
@dataclass
class Kontakt:
    fornavn: str
    etternavn: str
    telefon: int = None
    mobil: int = None
    adresse: str = None
    postnummer: int = None
    poststed: str = None
    epost: str = None


# Denne klassen kan vi bruke på samme måte som Kontakt-klassen vi definerte tidligere:

kontakt = Kontakt(
    fornavn="Bob",
    etternavn="Sandnes",
    epost="bob.sandnes@domene.no",
    telefon=51265854,
    adresse="Langgata 1",
    postnummer=4306,
    poststed="Sandnes",
    mobil=40392284,
)
kontakt2 = Kontakt(
    "Carl",
    "Hansen",
    22334455,
    99887766,
    "Kortveien 2",
    5000,
    "Bergen",
    "carl.hansen@domene.no",
)
kontakt3 = Kontakt("Dina", "Larsen")

kontakter = [kontakt, kontakt2, kontakt3]
print("Kontakter:")
for k in kontakter:
    print(f"{k.fornavn} {k.etternavn}, epost: {k.epost}")

## 3.1.8: Nøstede dataklasser

Vi kan definere flere dataklasser som videre klassifiserer og samler data, for eksempel ved å lage en egen klasse for Adresse.


In [ ]:
@dataclass
class Adresse:
    gate: str
    husnummer: str
    postnummer: int
    poststed: str
    land: str | None = None


@dataclass
class Kontakt:
    fornavn: str
    etternavn: str
    telefon: int | None = None
    mobil: int | None = None
    adresse: Adresse | None = None
    epost: str | None = None


kontakt = Kontakt(
    fornavn="Eva",
    etternavn="Nilsen",
    telefon=33445566,
    mobil=99887766,
    adresse=Adresse(
        gate="Hovedgata",
        husnummer="10B",
        postnummer=1234,
        poststed="Oslo",
        land="Norge",
    ),
    epost="eva.nilsen@example.com",
)

***

## 3.1.9: Oppsummering

Vi har nå sett litt på ulike måter å håndtere og organisere data på i python,
med bruk av klasser og `dataclass`. Når vi jobber med data er det viktig å ha
kontroll på datatyper, og sørge for konvertering og validering av data. Vi har
sett litt på hvordan det kan skjære seg når vi ikke har robust kode som
validerer at data faktisk har forventet format når vi behandler den. Uten
validering og feilhåndtering kan vi lett få feil som enten leder til at
programmet krasjer, eller at vi får feilaktige resultater. Mangel på validering
og feilhåndtering kan også gjøre det vanskelig å spore feil som oppstår under
kjøring av programmet.

I `Kontakt` klassen vår over, for eksempel, har vi ingen validering av data.
Det vil si at det gjøres ingen kontroller av om `kontakt.adresse` inneholder et
`Adresse` objekt, eller en tekststreng. Det gjøres heller ikke sjekk om `telefon`
er lagret som `str` eller `int`. Dette kan være akseptabelt om du har god kontroll
på data som kommer inn, men dersom du skal hente data gjennom brukerinput eller et
api, kan det være lurt å implementere en form for validering.

Det finnes et bibliotek til python som gjør det enklere å validere og konvertere
data, som gjør det ypperlig til bruk sammen med json-data fra api-er. Det skal
vi se litt nærmere på nå!

***

## Oppgaver (3.1-4 -> 3.1-5) - Definer egne (data)klasser

### Oppgave 3.1-4

Her skal du definere noen egne klasser. Velg deg først ut noe valgfri data
som du skal klassifisere (det kan være hva som helst). 

Først skal du definere minst to vanlige klasser (ikke `dataclass`), med
relevante egenskaper for data du har valgt deg ut. Datastrukturen
skal være nøstet, altså skal minst en av egenskapene ha type som en 
annen klasse du har definert.

Initialiser et par objekter av klassene, med eksempelverdier.

In [ ]:
## Din kode her

### Oppgave 3.1-5

Nå skal du redefinere de samme objektene, men bruke `dataclass` i stedet
for en vanlig klasse-definisjon.

Initialiser et par objekter av klassene, med eksempelverdier.

In [ ]:
## Din kode her